# Find all fits files associated with a given target
This code will read in a csv file of RA,DEC coordinates and the star name (same file used as our input catalog on Lesedi) and then search the header of all fits files in all subdirectories, and output a list of fits files which observed a given object.  

In [ ]:

import os
from pathlib import Path
import glob
import numpy as np
import shutil
from astropy.io import fits
from astropy.io import ascii
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.coordinates import match_coordinates_sky
from astropy.coordinates import FK5

#find all of the processed fits files in subdirectories
my_cwd = Path(os.getcwd())
fits_files = list(my_cwd.glob('**/fbMKD*.fits'))
#come back as a random list, so sort it
fits_files.sort()

#print(fits_files)

#read in the list of target stars
target_stars = ascii.read('dartmouth.csv', format='basic', delimiter=',', guess=False) 

#convert ra and dec strings into a Sky Coord object
target_coords = SkyCoord(ra=target_stars['ra'], dec=target_stars['dec'], unit=(u.hourangle, u.deg))

#print(target_coords[2].ra.hms, target_coords[2].dec.dms)


## Using a 2-d array for the file names, with an separate index dictonary

In [ ]:
#create a dictionary, where the key is the star name and the value is the index number
target_names = {}
for x in range(0, len(target_stars)):
    starname = target_stars["name"][x]
    target_names[starname] = x
    
#print(target_names)

#create empty 2-D arrays which will be used to store the filenames for each star in g and i
target_filesg = [list() for f in range(len(target_stars))]   
target_filesi = [list() for f in range(len(target_stars))]   
#print(target_files)      
        
science_keyword = "SCIENCE"
    
for file in fits_files:
    hdul = fits.open(file)  # open a FITS file
    hdr = hdul[0].header  #extract the header information 
#    print(file)

    if hdr['IMAGETYP'] == science_keyword and 'OBJECT'  in hdr  :
        ra_obs = hdr['RA']
        dec_obs = hdr['DEC']
        time_obs = hdr['DATE-OBS']
#        print("image header: ", ra_obs,dec_obs)
        observed_coords = SkyCoord(ra=ra_obs, dec=dec_obs,frame=FK5, equinox=time_obs, unit=(u.hourangle, u.deg))
#match the header object coordinates to the target star coordinates
        idx, d2d, d3d = match_coordinates_sky(observed_coords, target_coords)
#idx contains the index number of our star, so let's get its name        
        match2 = target_stars["name"][idx]
#        print("match key:", match2)
#note that file contains the full POXIS path of the filen
        file_name = file
# get just the name of the file we are currently searching       
#        file_name2 = file.name
#get the name of the subfolder
#        file_name_components = file.as_posix().split('/')
#        my_indx = len(file_name_components) - 2
# create a string which contains the name of the subfolder and the file name
#        file_name = "./" + file_name_components[my_indx] + "/" + file_name2

    # add the file name to our 2-D array
        if hdr['FILTERA'] == 'g' :
               target_filesg[target_names[match2]].append(file_name)
        elif hdr['FILTERA'] == 'i' :
               target_filesi[target_names[match2]].append(file_name)
        else :
               print("File is not a g or i filter: ", file_name,  hdr['FILTERA'] )
    

#print things out
for x in range(1, len(target_stars)):
    starname = target_stars["name"][x] 
    if len(target_filesi[x]) > 0 : 
        print(starname,"g", *target_filesg[x],sep=', ')   
        print(starname,"i", *target_filesi[x],sep=', ') 
        print()
   


## Move files to seperate subfolders for each star in each filter

In [ ]:
######################################################################################

# move a file to a specific subdirectory
def move_files(new_dir, file_list):
    
    for file in file_list : 
        shutil.move(file, new_dir)
        print('Moved:', file)


####################################################################################

#create a directory
def create_dir(starname):

    new_folder = starname + "/"
    
    p = Path(new_folder)
#    print(p)
    p.mkdir(parents=True, exist_ok=True)
    
#    try:
#        os.mkdir(path)
#    except OSError:
#        print ("Creation of the directory %s failed" % path)
#    else:
#        print ("Successfully created the directory %s " % path)

####################################################################################

for x in range(0, len(target_stars)):
    starname = target_stars["name"][x]
    if len(target_filesg[x]) > 0 :
        dir_name = starname + "g"
        print("dir_name:", dir_name)
        create_dir(dir_name)
        move_files(dir_name,target_filesg[x] )
    if len(target_filesi[x]) > 0 :
        dir_name = starname + "i"
        print("dir_name:", dir_name)
        create_dir(dir_name)
        move_files(dir_name,target_filesi[x] )